## Sistema de recomendación de peliculas

Cuando se necesita recomendar peliculas a un usuario, necesitamos encontar usuarios similares en el conjunto de datos y luego llegar a recomendaciones.

In [1]:
import json
import numpy as np
from compute_scores import pearson_score
from collaborative_filtering import find_similar_users

In [11]:
#Obtener las peliculas recomendadas de un usuario
def get_recommendations(dataset, input_user):
    if input_user not in dataset:
        raise TypeError('No se encontro el '+input_user+' in los datos')
    overall_scores = {}
    similarity_scores = {}

    #Calculamos el puntaje de similitud entre el usuario de entrada y los demas usuarios
    for user in [x for x in dataset if x != input_user]:
        similarity_score = pearson_score(dataset, input_user, user)

        if similarity_score <= 0:
            continue
        #extrae las peliculas que han sido calificadas por los otros usuarios pero no por el usuario de entrada 
        filtered_list = [x for x in dataset[user] if x not in  dataset[input_user] or dataset[input_user][x] == 0]
        for item in filtered_list:
            overall_scores.update({item: dataset[user][item]* similarity_score})
            similarity_scores.update({item: similarity_score})
    
    if len(overall_scores) == 0:
        return ['No se encontraron recomendaciones']
    
    movie_scores = np.array([[score/similarity_scores[item], item] for item, score in overall_scores.items()])

    #ordena la partituras y extrae las recomendaciones de peliculas
    movie_scores = movie_scores[np.argsort(movie_scores[:,0])[::-1]]
    movie_recommendations = [movie for _, movie in movie_scores]

    return movie_recommendations


In [12]:
def salida(dataset, user):
    print("\nPeliculas recomendadas para "+user+":")
    movies =get_recommendations(dataset,user)
    for i, movie in enumerate(movies):
        print(str(i+1)+ '. '+movie)

In [13]:
ratings_file = 'ratings.json'
with open(ratings_file, 'r') as f:
    data = json.loads(f.read())

salida(data, "Chris Duncan")


Peliculas recomendadas para Chris Duncan:
1. Vertigo
2. Scarface
3. Goodfellas
4. Roman Holiday
